In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [2]:
# Load Dataset
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.drop('Unnamed: 0', axis=1, inplace=True)
test_df = test_df.rename(columns={'id': 'file_name'})

In [3]:
train_df, val_df = train_test_split(train_df, test_size=0.05, random_state=42, stratify=train_df['label'])

print(f'Train shape: {train_df.shape}')
print(f'Validation shape: {val_df.shape}')
print(f'Test shape: {test_df.shape}')

Train shape: (75952, 2)
Validation shape: (3998, 2)
Test shape: (5540, 1)


In [4]:
# Define data augmentation with stronger variations
train_transforms = transforms.Compose([
    transforms.Resize((300, 300)),  
    transforms.RandomResizedCrop(300, scale=(0.6, 1.0)),  
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        return image, label

In [7]:
# Create datasets
train_dataset = CustomImageDataset(train_df, transform=train_transforms)
val_dataset = CustomImageDataset(val_df, transform=val_test_transforms)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=0, pin_memory=True)


In [8]:
device = torch.device("cpu")  # CPU training

# Load EfficientNet-B3 without pretrained weights
model = models.efficientnet_b3(weights=None)  # No pretrained weights as per competition rule
model.classifier = nn.Sequential(
    nn.Linear(model.classifier[1].in_features, 1),
    nn.Sigmoid()
)
model = model.to(device)

# Define Optimizer & Loss Function
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.BCEWithLogitsLoss()  

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=50)


In [9]:
# Gradient Accumulation Settings
accumulation_steps = 4  
epochs = 12  
best_f1 = 0

# Training Loop
for epoch in range(epochs):
    model.train()
    train_loss, train_accuracy = 0.0, 0.0
    optimizer.zero_grad()

    for i, (data, label) in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}")):
        data, label = data.to(device), label.float().unsqueeze(1).to(device)

        output = model(data)
        loss = criterion(output, label)
        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        train_loss += loss.item()
        preds = (output >= 0.5).float()
        train_accuracy += (preds == label).float().mean().item()

    train_loss /= len(train_loader)
    train_accuracy /= len(train_loader)

    print(f"Epoch {epoch+1}/{epochs} - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")

    # Validation Loop
    model.eval()
    val_pred_classes, val_labels_list = [], []

    with torch.no_grad():
        for data, label in tqdm(val_loader, desc="Validation"):
            data, label = data.to(device), label.float().unsqueeze(1).to(device)
            output = model(data)
            preds = (output >= 0.5).float()
            val_pred_classes.extend(preds.cpu().numpy().flatten())
            val_labels_list.extend(label.cpu().numpy().flatten())

    val_f1 = f1_score(val_labels_list, val_pred_classes, average='binary')
    print(f"Validation F1 Score: {val_f1:.4f}")

    # Save best model
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), "best_efficientnetb3_model.pth")
        print("Best model saved!")

    scheduler.step()  # Adjust learning rate


Training Epoch 1: 100%|██████████████████████████████████████████████████████████| 4747/4747 [8:21:39<00:00,  6.34s/it]


Epoch 1/12 - Loss: 0.6179, Accuracy: 0.7393


Validation: 100%|████████████████████████████████████████████████████████████████████| 250/250 [07:00<00:00,  1.68s/it]


Validation F1 Score: 0.8044
Best model saved!


Training Epoch 2: 100%|██████████████████████████████████████████████████████████| 4747/4747 [8:42:26<00:00,  6.60s/it]


...

Epoch 4/12 - Loss: 0.5872, Accuracy: 0.8124


Validation: 100%|████████████████████████████████████████████████████████████████████| 250/250 [07:01<00:00,  1.69s/it]


Validation F1 Score: 0.7542


Training Epoch 5:   0%|                                                             | 5/4747 [00:37<9:45:35,  7.41s/it]


KeyboardInterrupt: 

In [10]:
# Load Best Model for Testing
model.load_state_dict(torch.load("best_efficientnetb3_model.pth"))
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [12]:
# Test Dataset Class
class TestImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image


In [13]:
# Test DataLoader
test_dataset = TestImageDataset(test_df, transform=val_test_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)

In [14]:
# Generate Test Predictions
test_pred_classes = []

with torch.no_grad():
    for data in tqdm(test_loader, desc="Generating Test Predictions"):
        data = data.to(device)
        output = model(data)
        predicted_classes = (output >= 0.5).int()
        test_pred_classes.extend(predicted_classes.cpu().numpy().flatten())

Generating Test Predictions: 100%|███████████████████████████████████████████████████| 174/174 [11:25<00:00,  3.94s/it]


In [15]:
# Prepare Submission File
submission_df = pd.DataFrame({"id": test_df['file_name'], "label": test_pred_classes})
submission_df.to_csv("ai_vs_human_submission_efficientnetb3.csv", index=False)

print("Submission file saved successfully!")

Submission file saved successfully!


In [16]:
submission_df.label.value_counts()

label
0    2783
1    2757
Name: count, dtype: int64